In [2]:
# importing the packages necesary

import torch
from torch import nn
import pandas as pd
import numpy as np

## Model 0 - Prelims, using torch to build linear regression model



In [3]:
# building the model using torch

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        # linear function
        self.model = nn.Linear(input_dim, output_dim)


    def forward(self, X):
        out = self.model(X)
        return out

In [5]:
model = LinearRegressionModel(input_dim = 1,  output_dim = 1)

In [6]:
# setting the parameters
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

## example of a formula it's trying to learn

$$
y = f(x;\theta)
$$

where $f(.)$ is $3.4x_1 + 2.5$


In [7]:
x = torch.rand(1000,1)
y = x * 3.4 + 2.5

data = torch.cat((x,y), 1)

In [8]:
# prepare the dataset

batch_size = 10
n_epochs = 100

train_loader = torch.utils.data.DataLoader(dataset = data,
                                           batch_size = 10,
                                           shuffle = True)

In [9]:
for epochs in range(n_epochs):
    for i, _data in enumerate(train_loader):

        _x = _data.T[0].reshape(-1, 1)
        _y = _data.T[-1].reshape(-1, 1)

        # zero the gradients
        optimizer.zero_grad()

        outputs = model(_x)
        loss = loss_fn(outputs, _y)

        loss.backward()

        optimizer.step()


In [10]:
model.state_dict()

OrderedDict([('model.weight', tensor([[3.4000]])),
             ('model.bias', tensor([2.5000]))])